# Continuous Control

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

In [1]:
import gym
import random
import torch
import numpy as np
from collections import deque
import matplotlib.pyplot as plt
%matplotlib inline

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [2]:
env = gym.make('BipedalWalker-v2')

random_seed = 10
env.seed(random_seed)

# from agent import Agent
# agent = Agent(state_size=env.observation_space.shape[0], action_size=env.action_space.shape[0], 
#               random_seed=random_seed)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


/home/arasdar/anaconda3/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


[10]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [3]:
env.action_space, env.observation_space

(Box(4,), Box(24,))

In [4]:
action_size = env.action_space.shape[0]
state_size = env.observation_space.shape[0]

In [5]:
action_size, state_size

(4, 24)

In [6]:
env.observation_space.low, env.observation_space.high

(array([-inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf, -inf,
        -inf, -inf], dtype=float32),
 array([inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf,
        inf, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf],
       dtype=float32))

In [7]:
env.action_space.low, env.action_space.high

(array([-1., -1., -1., -1.], dtype=float32),
 array([1., 1., 1., 1.], dtype=float32))

### 3. Instantiate DDPG Agent

In [8]:
from agent2 import Agent

agent = Agent(s_size=state_size, a_size=action_size, random_seed=random_seed)

### 4. Train the Agent with DDPG

In [10]:
from collections import deque
import numpy as np
import time
import torch

n_episodes=1000
# n_steps=1000
n_learn=1
# learn_every=20
R_goal=0 # TOCHECK
Rs_deque = deque(maxlen=100)
meanRs = [] # plot
Rs = [] # plot
dlosses = []
glosses = []

In [ ]:
for i_episode in range(n_episodes):
    
    # # Reset Env
    # env_info = env.reset(train_mode=train_mode)[brain_name]       # reset the environment    
    # S = env_info.vector_observations                   # get the current state (for each agent)
    s = env.reset() # s: state [-1:1, -1:1, -8:8]
    #print(s.shape)

    #Rs = np.zeros(num_agents)                            # initialize the score (for each agent)
    R = 0 # R: total reward
    
    glosses_batch = []
    dlosses_batch = []

    #for i_step in range(n_steps):
    while True:
        s = np.reshape(s, [1, -1])
        a = agent.act(s) # a=[-2, 2]
        #a *= 2
        #print(a.shape)
        
        # Exploration noise
        # noise = np.random.normal(loc=0, scale=0.1, size=[num_agents, action_size])
        # A += noise
        # A = np.clip(a=A, a_min=-1, a_max=1)

        # env_info = env.step(A)[brain_name]            # send all actions to the environment
        # S2 = env_info.vector_observations          # get next state (for each agent)
        # rewards = env_info.rewards                          # get reward (for each agent)
        # dones = env_info.local_done                         # see if episode finished
        #s2: next_state, r: reward, done/terminal, env: environment
        a = np.reshape(a, [-1])
        s2, r, done, _ = env.step(a)
        #print(s2.shape, r, done)

        #s2_, q_ = agent.env(s, a)
        a = np.reshape(a, [1, -1])
        #s2_, _ = agent.env(s, a)
        # print(s2_)
        # s2_ = np.tanh(s2_) # predicted next state [-1:1, -1:1, -8:8]
        # print(s2_)
        # s2_[2] = s2_[2] * 8
        # print(s2_)

        # for s, a, reward, s2, done in zip(S, A, rewards, S2, dones):
        # agent.step(state, action, reward, next_state, done) # send actions to the agent
        agent.step(s, a, r, s2, done)

        R += r                           # update the score (for each agent)
        s = s2                                # roll over the states to the next state

        # if i_step%learn_every == 0:
        dlosses_Nlearn = []
        for _ in range(n_learn):
            dloss = agent.start_dlearn()
            #print(dloss)
            dlosses_Nlearn.append(dloss)
        dlosses_batch.append(np.mean(dlosses_Nlearn))
            
        #print(len(agent.memory))
        gloss = agent.start_glearn()
        glosses_batch.append(gloss)

        # if np.any(dones):                                   # exit loop if episode finished
        #     break
        if done:
            break

    #meanR = np.mean(Rs)
    #Rs_deque.append(meanR)
    Rs_deque.append(R)
    meanRs.append(np.mean(Rs_deque)) # plot
    Rs.append(R) # plot
    dlosses.append(np.mean(dlosses_batch))
    glosses.append(np.mean(glosses_batch))

    print('\rEpisode {}\tTotal Average Score: {:.2f}\tGloss: {:.2f}, \tDloss: {:.2f}'.format(
           i_episode+1, np.mean(Rs_deque), np.mean(glosses_batch), np.mean(dlosses_batch)))
    
    if np.mean(Rs_deque) >= R_goal:
        torch.save(agent.g.state_dict(), 'g-bipedal.pth')
        torch.save(agent.d.state_dict(), 'd-bipedal.pth')
        break

Episode 1	Total Average Score: -99.62	Gloss: 0.00, 	Dloss: 0.00
Episode 2	Total Average Score: -99.65	Gloss: 0.00, 	Dloss: 0.00
Episode 3	Total Average Score: -99.69	Gloss: 0.00, 	Dloss: 0.00
Episode 4	Total Average Score: -102.80	Gloss: 0.00, 	Dloss: 0.00
Episode 5	Total Average Score: -104.62	Gloss: 0.00, 	Dloss: 0.00
Episode 6	Total Average Score: -103.78	Gloss: 0.00, 	Dloss: 0.00
Episode 7	Total Average Score: -104.27	Gloss: 1.09, 	Dloss: 22.94
Episode 8	Total Average Score: -115.67	Gloss: 1.92, 	Dloss: 5.04
Episode 9	Total Average Score: -118.09	Gloss: 2.80, 	Dloss: 6.21
Episode 10	Total Average Score: -122.42	Gloss: 2.99, 	Dloss: 7.94
Episode 11	Total Average Score: -123.75	Gloss: 2.82, 	Dloss: 6.84
Episode 12	Total Average Score: -123.75	Gloss: 2.66, 	Dloss: 6.33
Episode 13	Total Average Score: -120.36	Gloss: 2.13, 	Dloss: 5.60
Episode 14	Total Average Score: -120.84	Gloss: 2.13, 	Dloss: 4.46
Episode 15	Total Average Score: -120.48	Gloss: 2.27, 	Dloss: 4.48
Episode 16	Total Aver

Episode 121	Total Average Score: -130.47	Gloss: -143.85, 	Dloss: 26.43
Episode 122	Total Average Score: -130.33	Gloss: -143.85, 	Dloss: 26.34
Episode 123	Total Average Score: -130.19	Gloss: -144.03, 	Dloss: 21.79
Episode 124	Total Average Score: -129.97	Gloss: -144.32, 	Dloss: 24.92
Episode 125	Total Average Score: -129.42	Gloss: -144.80, 	Dloss: 27.42
Episode 126	Total Average Score: -129.33	Gloss: -144.66, 	Dloss: 25.00
Episode 127	Total Average Score: -129.21	Gloss: -144.95, 	Dloss: 21.85
Episode 128	Total Average Score: -128.88	Gloss: -145.01, 	Dloss: 25.75
Episode 129	Total Average Score: -128.86	Gloss: -145.37, 	Dloss: 26.18
Episode 130	Total Average Score: -128.76	Gloss: -145.61, 	Dloss: 25.23
Episode 131	Total Average Score: -128.58	Gloss: -146.03, 	Dloss: 27.20
Episode 132	Total Average Score: -128.54	Gloss: -146.44, 	Dloss: 27.95
Episode 133	Total Average Score: -128.22	Gloss: -146.89, 	Dloss: 28.30
Episode 134	Total Average Score: -128.07	Gloss: -146.49, 	Dloss: 24.19
Episod

Episode 237	Total Average Score: -119.31	Gloss: -131.30, 	Dloss: 20.38
Episode 238	Total Average Score: -119.36	Gloss: -131.11, 	Dloss: 19.56
Episode 239	Total Average Score: -119.53	Gloss: -130.79, 	Dloss: 23.82
Episode 240	Total Average Score: -119.82	Gloss: -130.30, 	Dloss: 20.11
Episode 241	Total Average Score: -120.10	Gloss: -130.09, 	Dloss: 20.28
Episode 242	Total Average Score: -120.28	Gloss: -129.93, 	Dloss: 19.39
Episode 243	Total Average Score: -120.40	Gloss: -129.35, 	Dloss: 20.98
Episode 244	Total Average Score: -120.73	Gloss: -128.86, 	Dloss: 19.89
Episode 245	Total Average Score: -121.10	Gloss: -128.40, 	Dloss: 20.32
Episode 246	Total Average Score: -121.23	Gloss: -127.97, 	Dloss: 20.82
Episode 247	Total Average Score: -121.32	Gloss: -127.67, 	Dloss: 19.35
Episode 248	Total Average Score: -121.36	Gloss: -127.35, 	Dloss: 18.89
Episode 249	Total Average Score: -121.51	Gloss: -126.97, 	Dloss: 21.00
Episode 250	Total Average Score: -121.48	Gloss: -126.53, 	Dloss: 18.96
Episod

Episode 353	Total Average Score: -120.61	Gloss: -84.01, 	Dloss: 15.94
Episode 354	Total Average Score: -120.84	Gloss: -83.28, 	Dloss: 15.89
Episode 355	Total Average Score: -120.74	Gloss: -82.83, 	Dloss: 15.12
Episode 356	Total Average Score: -121.10	Gloss: -82.32, 	Dloss: 16.49
Episode 357	Total Average Score: -121.59	Gloss: -81.54, 	Dloss: 15.33
Episode 358	Total Average Score: -122.02	Gloss: -80.65, 	Dloss: 15.49
Episode 359	Total Average Score: -122.39	Gloss: -79.93, 	Dloss: 14.61
Episode 360	Total Average Score: -122.44	Gloss: -79.28, 	Dloss: 17.04
Episode 361	Total Average Score: -122.32	Gloss: -79.20, 	Dloss: 15.23
Episode 362	Total Average Score: -122.63	Gloss: -78.63, 	Dloss: 14.35
Episode 363	Total Average Score: -122.64	Gloss: -77.77, 	Dloss: 14.50
Episode 364	Total Average Score: -122.73	Gloss: -77.88, 	Dloss: 14.42
Episode 365	Total Average Score: -122.68	Gloss: -77.51, 	Dloss: 14.07
Episode 366	Total Average Score: -122.84	Gloss: -76.91, 	Dloss: 16.73
Episode 367	Total Av

Episode 470	Total Average Score: -135.66	Gloss: -105.52, 	Dloss: 23.48
Episode 471	Total Average Score: -135.71	Gloss: -104.28, 	Dloss: 26.69
Episode 472	Total Average Score: -136.12	Gloss: -103.24, 	Dloss: 23.54
Episode 473	Total Average Score: -136.00	Gloss: -101.78, 	Dloss: 25.01
Episode 474	Total Average Score: -135.58	Gloss: -101.32, 	Dloss: 19.45
Episode 475	Total Average Score: -135.74	Gloss: -101.47, 	Dloss: 20.23
Episode 476	Total Average Score: -135.59	Gloss: -100.80, 	Dloss: 23.58
Episode 477	Total Average Score: -135.51	Gloss: -100.31, 	Dloss: 22.70
Episode 478	Total Average Score: -135.42	Gloss: -100.01, 	Dloss: 24.06
Episode 479	Total Average Score: -135.28	Gloss: -100.11, 	Dloss: 21.13
Episode 480	Total Average Score: -135.03	Gloss: -99.94, 	Dloss: 23.51
Episode 481	Total Average Score: -135.12	Gloss: -99.85, 	Dloss: 21.55
Episode 482	Total Average Score: -134.97	Gloss: -99.02, 	Dloss: 21.27
Episode 483	Total Average Score: -134.83	Gloss: -98.71, 	Dloss: 21.47
Episode 48

### 5. Plot the result

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1,1,1)

plt.plot(np.arange(1, len(meanRs)+1), meanRs)
plt.ylabel('total_average_scores')
plt.xlabel('Episode #')
plt.show()

### 6. Watch the agent running with saved weights

In [36]:
# Load the saved weights into Pytorch model
agent.g.load_state_dict(torch.load('g-bipedal.pth', map_location='cpu'))
agent.d.load_state_dict(torch.load('d-bipedal.pth', map_location='cpu'))

# env_info = env.reset(train_mode=False)[brain_name]     # reset the environment    
# S = env_info.vector_observations                  # get the current states (S) (for each agent)
s = env.reset() # s: state # get the current state (s) (for an agent)
s_ = np.zeros_like(s)# s_: predicted state # predict the current state (s) (by the agent)
# print(s.shape)

#Rs = np.zeros(num_agents)                          # initialize the total scores (Rs) (for each agent)
R = 0 # R:total reward # initialize the total score (R) (for an agent)

while True:
    #A = agent.act(S)                        # select actions (A) from loaded model agents
    s = np.reshape(s, [1, -1])
    s_ = np.reshape(s_, [1, -1])
    a = agent.act(s, s_) # a: [-1, +1]    # select action (a) from loaded model agent
    #a = agent.act(s)
    #print(s.shape, a.shape)
    
    # env_info = env.step(A)[brain_name]           # send all actions (A) to tne environment (env)
    # S2 = env_info.vector_observations         # get next states (S2) (for each agent)
    # rewards = env_info.rewards                         # get rewards (for each agent)
    # dones = env_info.local_done                        # see if the episode is done/finished (terminal)
    s2, r, done, _ = env.step(np.reshape(a, [-1]))
    #s2, r, done = env.step(a)
    
    #s2_, q_ = agent.env(s, a)
    s2_, _ = agent.env(s, a)
    
    # Rs += env_info.rewards                         # update the total scores (Rs) (for each agent)
    # S = S2                               # roll over current states (S) to next states (S2)
    R += r # update the total score (R) (for an agent)
    s = s2 # roll over current state (s) to next state (s2)
    s_ = s2_ # roll over the predcited current state (s) to the predicted next state (s2_)
    
    #if np.any(dones):                                  # exit loop if episode is done/finished
    if done: # exit loop if episode is done/finished (terminal)
        break
        
print('Average of total scores: {}'.format(R))

Average of total scores: -106.75700939138544


When finished, you can close the environment.

In [37]:
env.close()